# Trading Agent using Bayesian NetworksThis notebook demonstrates how to build a trading agent using Bayesian Networks. It fetches stock market data, processes it, and uses Bayesian inference for decision-making.

In [ ]:
import yfinance as yfimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as sns!pip install pgmpyfrom pgmpy.models import BayesianNetworkfrom pgmpy.estimators import MaximumLikelihoodEstimatorfrom pgmpy.inference import VariableElimination

## Data Collection

In [ ]:
ticker = 'AAPL'data = yf.download(ticker, start='2010-01-01', end='2023-12-31')data.to_csv(f'{ticker}_data.csv')

## Bayesian Network

In [ ]:
model = BayesianNetwork([    ('RSI_Cat', 'Price_Movement'),    ('Vol_Cat', 'Price_Movement'),    ('Vol_Chg', 'Price_Movement'),    ('Price_Movement', 'Action')])
data['Price_Movement'] = pd.to_numeric(data['Price_Movement'], errors='coerce')data['Action'] = data['Price_Movement'].apply(lambda x: 'Buy' if x > 0 else 'Sell' if x < 0 else 'Hold')train_data = data.sample(frac=0.8, random_state=42)test_data = data.drop(train_data.index)train_data = train_data[['RSI_Cat', 'Vol_Cat', 'Vol_Chg', 'Price_Movement', 'Action']]train_data['Vol_Chg'] = np.sign(data['Volume'].pct_change()).replace({1: 'Up', -1: 'Down', 0: 'No Change'})train_data['Vol_Chg'].fillna('No Change', inplace=True)train_data['Price_Movement'] = np.sign(data['Close'].pct_change())train_data['Price_Movement'] = train_data['Price_Movement'].replace({1: 'Up', -1: 'Down', 0: 'No Change'})train_data['Price_Movement'].fillna('No Change', inplace=True)test_data['Vol_Chg'] = np.sign(test_data['Volume'].pct_change()).replace({1: 'Up', -1: 'Down', 0: 'No Change'})test_data['Vol_Chg'].fillna('No Change', inplace=True)model.fit(train_data, estimator=MaximumLikelihoodEstimator)

## Evaluation

In [ ]:
def calculate_returns(model, test_data):    portfolio = 10000    holdings = 0    for idx, row in test_data.iterrows():        evidence = {            'RSI_Cat': row['RSI_Cat'],            'Vol_Cat': row['Vol_Cat'],            'Vol_Chg': row['Vol_Chg']        }        action = inference.map_query(variables=['Action'], evidence=evidence)['Action']        if action == 'Buy' and portfolio > 0:            holdings += portfolio / row['Close']            portfolio = 0        elif action == 'Sell' and holdings > 0:            portfolio += holdings * row['Close']            holdings = 0    final_value = portfolio + holdings * test_data.iloc[-1]['Close']    return final_valuemodel_returns = calculate_returns(model, test_data)buy_hold_returns = 10000 * (test_data.iloc[-1]['Close'] / test_data.iloc[0]['Close'])print(f'Model Returns: ${model_returns:.2f}')print(f'Buy & Hold Returns: ${buy_hold_returns:.2f}')